In [1]:
import os, sys, glob, re
import datetime as dt
import numpy as np
from matplotlib.dates import date2num
import h5py

# from modest_image import imshow
#from calendar import monthrange

In [2]:
sys.path.insert(0,'./mi_instrument')
from mi.instrument.kut.ek60.ooicore import zplsc_b

In [3]:
import db_diff

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [5]:
%matplotlib inline

## Set up range of date to load data

In [6]:
date_start = dt.date(2015,8,17)
date_end = dt.date(2015,10,17)
date_wanted = [(date_start+dt.timedelta(days=xx)).strftime('%Y%m%d') for xx in range((date_end-date_start).days + 1)]

In [7]:
data_path = '/media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/'
save_path = '/media/wu-jung/wjlee_apl_2/ooi_zplsc_new/'
save_fname = '%s-%s_new' % (date_start.strftime('%Y%m%d'),date_end.strftime('%Y%m%d'))

In [8]:
# Set param
ping_time_param_names = ["hour_all","min_all","sec_all"]
ping_time_param_vals = (range(24),range(20),range(0,60,5))
ping_time_param = dict(zip(ping_time_param_names,ping_time_param_vals))
ping_per_day = len(ping_time_param['hour_all'])*len(ping_time_param['min_all'])*len(ping_time_param['sec_all'])

ping_bin_range = 40
depth_bin_range = 10
tvg_correction_factor = 2

ping_per_day_mvbs = ping_per_day/ping_bin_range

## Compile data

Compile data by the following:
- Unpack .raw file
- Estimate noise and clean up data
- Extract designated pings
- Calculate MVBS

In [9]:
db_diff.raw2MVBS_daterange(date_wanted,data_path,save_path,save_fname,ping_time_param,ping_bin_range,depth_bin_range,tvg_correction_factor)

11:30:20  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150817-T000000.raw


db_diff.py:176: RuntimeWarning: invalid value encountered in log10
  tmp = 10*np.log10(np.ma.masked_less_equal(subtract,0))


-- New H5 file, create new dataset...
-- New H5 file, create new dataset...
11:30:56  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150818-T000000.raw
-- H5 file exists, append new data mtx...
-- H5 file exists, append new data mtx...
11:31:27  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150819-T000000.raw
-- H5 file exists, append new data mtx...
-- H5 file exists, append new data mtx...
11:31:58  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150820-T000000.raw
-- H5 file exists, append new data mtx...
-- H5 file exists, append new data mtx...
11:32:28  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150821-T000000.raw
-- H5 file exists, append new data mtx...
-- H5 file exists, append new data mtx...
11:32:58  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150822-T000000.raw
-- H5 file exists, append new data mtx...
-- H5 file exists, append new data mtx...
11:33:28  unpacking file: /media/w

KeyboardInterrupt: 

In [9]:
for iD,dd in zip(range(len(date_wanted)),date_wanted):

    # load files and get calibration params
    fname = glob.glob(os.path.join(data_path,'OOI-D%s*.raw' %dd))[0]
    particle_data, data_times, power_data, freq, bin_size, config_header, config_transducer = \
        zplsc_b.parse_echogram_file(fname)
    cal_params = db_diff.get_cal_params(power_data,particle_data,config_header,config_transducer)

    # swap sequence of 120 kHz and 38 kHz cal_params and data
    cal_params = [cal_params[fn] for fn in [1,0,2]]
    power_data = db_diff.get_power_data_mtx(power_data,freq)

    # clean data
    Sv_raw_tmp = np.ma.empty((power_data.shape))
    Sv_corr_tmp = np.ma.empty((power_data.shape))
    Sv_noise_tmp = np.ma.empty((power_data.shape[0:2]))
    for fn in range(power_data.shape[0]):
        noise_est,ping_bin_num = db_diff.get_noise(power_data[fn,:,:],bin_size,ping_bin_range,depth_bin_range)
        Sv_raw_tmp[fn,:,:],Sv_corr_tmp[fn,:,:],Sv_noise_tmp[fn,:] = \
                db_diff.remove_noise(power_data[fn,:,:],cal_params[fn],noise_est.min(),ping_bin_range,tvg_correction_factor)

    # set up indexing to get wanted pings
    dd = dt.datetime.strptime(dd,'%Y%m%d')
    time_wanted = [dt.datetime(dd.year,dd.month,dd.day,hh,mm,ss) for hh in hour_all for mm in min_all for ss in sec_all]
    idx_wanted = [db_diff.find_nearest_time_idx(data_times,tt,2) for tt in time_wanted]
    notnanidx = np.argwhere(~np.isnan(idx_wanted)).flatten()
    notnanidx_in_all = np.array(idx_wanted)[notnanidx].astype(int)

    # get data to be saved
    Sv_raw = np.ma.empty((Sv_raw_tmp.shape[0],Sv_raw_tmp.shape[1],ping_per_day))
    Sv_corr = np.ma.empty((Sv_raw_tmp.shape[0],Sv_raw_tmp.shape[1],ping_per_day))
    Sv_noise = np.ma.empty((Sv_raw_tmp.shape[0],Sv_raw_tmp.shape[1],1))
    
    Sv_raw[:,:,notnanidx] = Sv_raw_tmp[:,:,notnanidx_in_all]
    Sv_corr[:,:,notnanidx] = Sv_corr_tmp[:,:,notnanidx_in_all]
    Sv_noise[:,:,0] = Sv_noise_tmp
    ping_time = date2num(time_wanted)

    idx_save_mask = np.argwhere(np.isnan(idx_wanted))
    Sv_raw[:,:,idx_save_mask] = np.ma.masked
    Sv_corr[:,:,idx_save_mask] = np.ma.masked
    
    # save into h5 file
    sz = Sv_raw.shape
    f = h5py.File(os.path.join(save_path,'%s_Sv.h5' %save_fname),"a")
    if "Sv_raw" in f:  # if file alread exist and contains Sv mtx
        print '-- H5 file exists, append new data mtx...'
        # append new data
        sz_exist = f['Sv_raw'].shape  # shape of existing Sv mtx
        f['Sv_raw'].resize((sz_exist[0],sz_exist[1],sz_exist[2]+sz[2]))
        f['Sv_raw'][:,:,sz_exist[2]:] = Sv_raw
        f['Sv_corr'].resize((sz_exist[0],sz_exist[1],sz_exist[2]+sz[2]))
        f['Sv_corr'][:,:,sz_exist[2]:] = Sv_corr
        f['Sv_noise'].resize((sz_exist[0],sz_exist[1],sz_exist[2]+1))
        f['Sv_noise'][:,:,sz_exist[2]:] = Sv_noise
        f['ping_time'].resize((sz_exist[2]+sz[2],))
        f['ping_time'][sz_exist[2]:] = ping_time
    else:
        print '-- New H5 file, create new dataset...'
        # create dataset and save
        f.create_dataset("Sv_raw", sz, maxshape=(sz[0],sz[1],None), data=Sv_raw, chunks=True)
        f.create_dataset("Sv_corr", sz, maxshape=(sz[0],sz[1],None), data=Sv_corr, chunks=True)
        f.create_dataset("Sv_noise", (Sv_raw.shape[0],Sv_raw.shape[1],1), maxshape=(sz[0],sz[1],None), data=Sv_noise, chunks=True)
        f.create_dataset("ping_time", (sz[2],), maxshape=(None,), data=ping_time, chunks=True)
        f.create_dataset("depth_bin_size",data=bin_size)
    f.close()

    # get MVBS
    MVBS = db_diff.get_MVBS(Sv_corr,bin_size,ping_bin_range,depth_bin_range=5)
    
    # save into h5 file
    sz = MVBS.shape
    ping_time_MVBS = ping_time[0::ping_bin_range]  # get ping time every ping_bin_range for MVBS
    
    f = h5py.File(os.path.join(save_path,'%s_MVBS.h5' %save_fname),"a")
    if "MVBS" in f:  # if file alread exist and contains Sv mtx
        print '-- H5 file exists, append new data mtx...'
        # append new data
        sz_exist = f['MVBS'].shape  # shape of existing Sv mtx
        f['MVBS'].resize((sz_exist[0],sz_exist[1],sz_exist[2]+sz[2]))
        f['MVBS'][:,:,sz_exist[2]:] = MVBS
        f['ping_time'].resize((sz_exist[2]+sz[2],))
        f['ping_time'][sz_exist[2]:] = ping_time_MVBS
    else:
        print '-- New H5 file, create new dataset...'
        # create dataset and save
        f.create_dataset("MVBS", sz, maxshape=(sz[0],sz[1],None), data=MVBS, chunks=True)
        f.create_dataset("ping_time", (sz[2],), maxshape=(None,), data=ping_time_MVBS, chunks=True)
        f.create_dataset("depth_bin_size",data=depth_bin_range)
    f.close()

00:15:52  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150817-T000000.raw


db_diff.py:171: RuntimeWarning: invalid value encountered in log10
  tmp = 10*np.log10(np.ma.masked_less_equal(subtract,0))


-- H5 file exists, append new data mtx...
-- H5 file exists, append new data mtx...
00:16:23  unpacking file: /media/wu-jung/wjlee_apl_2/ooi_zplsc_600m/OOI-D20150818-T000000.raw


KeyboardInterrupt: 

In [ ]:
from sklearn import decomposition
import nimfa
from scipy import signal
